In [23]:
import numpy as np
from spec_id import Gen_spec
import matplotlib.pyplot as plt
from matplotlib import gridspec
from vtl.Readfile import Readfile
from vtl.Photo import Photometry
import pandas as pd
from glob import glob
from astropy.io import fits, ascii
from astropy.table import Table
from scipy.interpolate import interp1d
import os
import cPickle
import seaborn as sea
sea.set(style='white')
sea.set(style='ticks')
sea.set_style({"xtick.direction": "in","ytick.direction": "in"})
colmap = sea.cubehelix_palette(12, start=2, rot=.2, dark=0, light=1.1, as_cmap=True)

In [24]:
galDB = pd.read_pickle('../data/sgal_param_DB.pkl')
metal=np.arange(0.002,0.031,0.001)
age=np.arange(.5,6.1,.1)
tau=[0,8.0, 8.3, 8.48, 8.6, 8.7, 8.78, 8.85, 8.9, 8.95, 9.0, 9.04, 9.08, 9.11, 9.15, 9.18, 9.2, 9.23, 9.26, 9.28,
     9.3, 9.32, 9.34, 9.36, 9.38, 9.4, 9.41, 9.43, 9.45, 9.46, 9.48]

In [25]:
def Scale_model(mwv,mfl,swv,sfl,ser,rshift):
    imfl = interp1d(mwv * (1+rshift),mfl)(swv)
    C = np.sum(((sfl * imfl) / ser ** 2)) / np.sum((imfl ** 2 / ser ** 2))
    return C

In [5]:
goods_s=Table.read('../../../Clear_data/goodss_mosaic/goodss_3dhst.v4.3.cat',format='ascii')
goods_n=Table.read('../../../Clear_data/goodsn_mosaic/goodsn_3dhst.v4.3.cat',format='ascii')

In [14]:
Hs = Table.read('../../../../../../Volumes/Vince_research/goods_4.4/goodss_v4.4/goodss-F105W-astrodrizzle-v4.4_drz_sub.cat',format='ascii')
Hn = Table.read('../../../../../../Volumes/Vince_research/goods_4.4/goodsn_v4.4/goodsn-F105W-astrodrizzle-v4.4_drz_sub.cat',format='ascii')

In [6]:
cnames = list(goods_s.columns)

# for i in range(len(cnames)):
#     if cnames[i][0:2] == 'f_':
#         print cnames[i]
filt_s = ['f_F606Wcand','f_F606W','f_R','f_Rc','f_F775W','f_I','f_F814Wcand','f_F850LP','f_F850LPcand','f_F098M','f_F105W',
          'f_F125W','f_J','f_tenisJ','f_F140W','f_H','f_tenisK','f_Ks','f_IA624','f_IA651','f_IA679','f_IA709','f_IA738',
          'f_IA767','f_IA797','f_IA827','f_IA856']
e_filt_s = ['e_F606Wcand','e_F606W','e_R','e_Rc','e_F775W','e_I','e_F814Wcand','e_F850LP','e_F850LPcand','e_F098M','e_F105W',
          'e_F125W','e_J','e_tenisJ','e_F140W','e_H','e_tenisK','e_Ks','e_IA624','e_IA651','e_IA679','e_IA709','e_IA738',
            'e_IA767','e_IA797','e_IA827','e_IA856']

filt_n = ['f_F606W','f_R','f_Rs','f_I','f_F775W','f_Z','f_F850LP','f_F105W','f_F125W','f_J','f_F140W','f_H','f_Ks']
e_filt_n = ['e_F606W','e_R','e_Rs','e_I','e_F775W','e_Z','e_F850LP','e_F105W','e_F125W','e_J','e_F140W','e_H','e_Ks']

In [7]:
filters=['f_F606Wcand','f_F606W','f_R','f_Rc','f_F775W','f_I','f_F814Wcand','f_F850LP','f_F850LPcand','f_F098M','f_F105W',
          'f_F125W','f_J','f_tenisJ','f_F140W','f_H','f_tenisK','f_Ks','f_IA624','f_IA651','f_IA679','f_IA709','f_IA738',
          'f_IA767','f_IA797','f_IA827','f_IA856','f_F606W','f_R','f_Rs','f_I','f_F775W','f_Z','f_F850LP','f_F105W',
          'f_F125W','f_J','f_F140W','f_H','f_Ks']

f_errors=['e_F606Wcand','e_F606W','e_R','e_Rc','e_F775W','e_I','e_F814Wcand','e_F850LP','e_F850LPcand','e_F098M','e_F105W',
          'e_F125W','e_J','e_tenisJ','e_F140W','e_H','e_tenisK','e_Ks','e_IA624','e_IA651','e_IA679','e_IA709','e_IA738',
          'e_IA767','e_IA797','e_IA827','e_IA856','e_F606W','e_R','e_Rs','e_I','e_F775W','e_Z','e_F850LP','e_F105W',
          'e_F125W','e_J','e_F140W','e_H','e_Ks']

In [35]:
filt_names=['s_F606Wcand','s_F606W','s_R','s_Rc','s_F775W','s_I','s_F814Wcand','s_F850LP','s_F850LPcand','s_F098M','s_F105W',
          's_F125W','s_J','s_tenisJ','s_F140W','s_H','s_tenisK','s_Ks','s_IA624','s_IA651','s_IA679','s_IA709','s_IA738',
          's_IA767','s_IA797','s_IA827','s_IA856','n_F606W','n_R','n_Rs','n_I','n_F775W','n_Z','n_F850LP','n_F105W',
          'n_F125W','n_J','n_F140W','n_H','n_Ks']
effwv = [5818,5818,6418,6460,7634,8549,8001,9001,9001,9841,10476,12410,12311,12513,13831,16451,21534,21638,
         6229,6497,6780,7072,7358,7679,7964,8245,8563,5818,6487,6788,7939,7634,9077,9001,10476,12410,12499,13831,16307,21541]
fwhm = [1550,1550,943,1103,990,1356,1585,1235,1235,1163,1979,2041,1774,1080,2676,1991,2135,1854,225,240,250,
        237,237,262,266,245,266,1550,817,1047,982,990,970,1235,1979,2041,1108,2676,1912,2073]
phot_db = pd.DataFrame({'eff_wv' : effwv, 'fwhm' : fwhm },index = filt_names)

In [36]:
phot_db

,eff_wv,fwhm
s_F606Wcand,5818,1550
s_F606W,5818,1550
s_R,6418,943
s_Rc,6460,1103
s_F775W,7634,990
s_I,8549,1356
s_F814Wcand,8001,1585
s_F850LP,9001,1235
s_F850LPcand,9001,1235
s_F098M,9841,1163


In [37]:
for i in galDB.index:
    f_phot = np.repeat(-99.0,len(filters))
    f_err = np.repeat(-99.0,len(filters))
    if galDB['gids'][i][0] == 's':
        for ii in range(len(filt_s)):
            if filt_s[ii] == 'f_F105W':
                f_phot[ii] = 10**((25-Hs[galDB['ids'][i]-1]['MAG_AUTO'])/2.5)
                f_err[ii] = 10**((25-Hs[galDB['ids'][i]-1]['MAG_AUTO'])/2.5) * Hs[galDB['ids'][i]-1]['FLUXERR_AUTO']/Hs[galDB['ids'][i]-1]['FLUX_AUTO']
            else:
                f_phot[ii] = goods_s[galDB['ids'][i]-1][filt_s[ii]]
                f_err[ii] = goods_s[galDB['ids'][i]-1][e_filt_s[ii]]

    if galDB['gids'][i][0] == 'n':
        for ii in range(len(filt_n)):
            if filt_n[ii] == 'f_F105W':
                f_phot[ii + len(filt_s)] = 10**((25-Hn[galDB['ids'][i]-1]['MAG_AUTO'])/2.5)
                f_err[ii + len(filt_s)] = 10**((25-Hn[galDB['ids'][i]-1]['MAG_AUTO'])/2.5) * Hn[galDB['ids'][i]-1]['FLUXERR_AUTO']/Hn[galDB['ids'][i]-1]['FLUX_AUTO']
            else:
                f_phot[ii + len(filt_s)] = goods_n[galDB['ids'][i]-1][filt_n[ii]]
                f_err[ii + len(filt_s)] = goods_n[galDB['ids'][i]-1][e_filt_n[ii]]
    
    phot_add = pd.DataFrame({galDB['gids'][i] : f_phot, '%s_err' % galDB['gids'][i] : f_err },index = filt_names)
    phot_db = phot_db.join(phot_add)

In [39]:
phot_db

,eff_wv,fwhm,n14713,n14713_err,n16758,n16758_err,n17070,n17070_err,n19442,n19442_err,...,s45972,s45972_err,s46066,s46066_err,s46345,s46345_err,s47677,s47677_err,s48631,s48631_err
s_F606Wcand,5818,1550,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000
s_F606W,5818,1550,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,1.842122,0.042595,0.508798,0.028743,-99.000000,-99.000000,-99.000000,-99.000000,1.360973,0.075360
s_R,6418,943,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,2.896026,0.027874,0.691281,0.026387,0.250015,0.018163,0.820523,0.024696,2.152800,0.051871
s_Rc,6460,1103,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,2.990469,0.069154,0.773933,0.051897,0.193690,0.042023,0.816439,0.053637,2.075598,0.096157
s_F775W,7634,990,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,6.765190,0.072645,2.371791,0.048622,-99.000000,-99.000000,-99.000000,-99.000000,4.521061,0.129567
s_I,8549,1356,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,13.561190,0.423599,4.134985,0.293417,1.334638,0.257392,3.297968,0.296045,9.430043,0.467359
s_F814Wcand,8001,1585,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,8.995494,0.064019,3.079942,0.038375,0.782762,0.071825,-99.000000,-99.000000,6.211490,0.124296
s_F850LP,9001,1235,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,15.831162,0.095238,5.559113,0.067928,-99.000000,-99.000000,-99.000000,-99.000000,10.484480,0.168063
s_F850LPcand,9001,1235,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,15.894367,0.313434,5.517749,0.138862,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000
s_F098M,9841,1163,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,22.564123,0.086817,9.745344,0.054790,-99.000000,-99.000000,-99.000000,-99.000000,15.307628,0.135199


In [20]:
phot_db.to_pickle('../data/phot_db.pkl')

In [26]:
phot_db = pd.read_pickle('../data/phot_db.pkl')

In [8]:
galDB

,gids,ids,ra,dec,hi_res_specz,Z,Z_-_sig,Z_+_sig,t,t_-_sig,t_+_sig,lmass
43,n14713,14713,189.236333,62.214608,1.232,0.019,0.012,0.008,3.1,0.903,0.541,10.56
49,n16758,16758,189.162357,62.224840,1.014,0.021,0.002,0.002,5.5,0.470,0.125,10.83
50,n17070,17070,189.268086,62.226445,1.192,0.017,0.007,0.006,2.5,0.732,0.998,10.92
58,n19442,19442,189.123036,62.237460,1.101,0.015,0.010,0.010,2.9,1.033,0.818,10.10
61,n21156,21156,189.239409,62.247548,1.251,0.015,0.005,0.006,2.6,0.524,0.622,11.16
62,n21427,21427,189.368121,62.247344,1.514,0.020,0.009,0.007,2.8,0.779,0.533,10.73
69,n23435,23435,189.338306,62.256571,1.134,0.015,0.007,0.008,3.7,0.907,0.779,10.27
86,n32566,32566,189.161280,62.307577,1.150,0.020,0.008,0.007,2.7,1.054,0.919,10.61
96,n34694,34694,189.147840,62.323647,1.145,0.014,0.005,0.006,3.1,0.671,1.048,10.94
107,n37686,37686,189.274474,62.360820,1.258,0.016,0.003,0.003,3.6,0.598,0.548,10.89


In [32]:
def Plot_with_phot(galaxy,redshift, bfmetal, bfage, tau, save_plot=False,plot_name=''):
    
    spec = Gen_spec(galaxy, redshift)
    # spec.Median_spec(bfmetal,bfage,tau)
    # mwv,mfl=np.load('../../../fsps_models_for_fit/fsps_spec/m%s_a%s_t%s_spec.npy' % (spec.bfmetal,spec.bfage,spec.bftau))
    
    Clam = 3E18 / phot_db['eff_wv'] **2 * 10**((-1.1)/2.5-29)
    # Cm = Scale_model( mwv, mfl, spec.gal_wv, spec.gal_fl, spec.gal_er, redshift)
    
    # filters = [240,201,197,198,284]
    # effwv = np.repeat(-99.,len(filters))
    # phot = np.repeat(-99.,len(filters))
    # photer = np.repeat(-99.,len(filters))
    # fwhm =np.repeat(-99.,len(filters))
    # 
    # gal_phot = Photometry(spec.gal_wv,spec.gal_fl,spec.gal_er,240)
    # 
    # for i in range(len(filters)):
    #     gal_phot.Get_Sensitivity(filters[i])
    #     if spec.gal_wv[0] <= gal_phot.sens_wv[0] and gal_phot.sens_wv[-1] <= spec.gal_wv[-1]:
    #         gal_phot.Photo()
    #         gal_phot.FWHM()
    #         effwv[i],phot[i],photer[i] = [gal_phot.eff_wv,gal_phot.photo,gal_phot.photo_er]
    #         fwhm[i] = gal_phot.fwhm/2
    
    ### get maximum y value for plot 
    if max(phot_db[galaxy]*1E18 * Clam) > max(spec.gal_fl*1E18):
        ymax = max(phot_db[galaxy]*1E18 * Clam)
    else:
        ymax = max(spec.gal_fl*1E18)
    
    gs=gridspec.GridSpec(2,1)

    plt.figure(figsize=[15,8])
    plt.subplot(gs[0])
    # plt.plot(mwv * (1 + redshift), mfl*Cm *1E18, color='k', alpha = .3)
    plt.plot(spec.gal_wv,spec.gal_fl * 1E18,color = 'k')
    plt.errorbar(phot_db['eff_wv'],phot_db[galaxy] * Clam *1E18 ,phot_db['%s_err' % galaxy] * Clam *1E18,
                 phot_db['fwhm']/2 ,color='#1D659D',fmt='D',zorder=5)
    # plt.errorbar(effwv,phot*1E18,photer*1E18,fwhm,color='#c90316',fmt ='s',zorder=5)
    # plt.xticks
    plt.ylim(0,ymax*1.1)
    plt.xlim(5800,22000)
    plt.minorticks_on()
    plt.ylabel('F$_\lambda$ (10$^{-18}$ erg/s/cm$^2$/$\AA$)',size=20)
    plt.tick_params(axis='both', which='major', labelsize=15)
    plt.title(galaxy,size=20)
    
    plt.subplot(gs[1])
    # plt.plot(mwv * (1 + redshift),mfl*Cm*1E18, color='k', alpha = .3)
    plt.plot(spec.gal_wv,spec.gal_fl * 1E18,color = 'k')
    plt.errorbar(phot_db['eff_wv'],phot_db[galaxy] * Clam *1E18 ,phot_db['%s_err' % galaxy] * Clam *1E18,
                 phot_db['fwhm']/2 ,color='#1D659D',fmt='D',zorder=5)
    # plt.errorbar(effwv,phot*1E18,photer*1E18,fwhm,color='#c90316',fmt ='s',zorder=5)
    plt.ylim(0,ymax*1.1)
    plt.xlim(7900,11300)
    plt.minorticks_on()
    plt.ylabel('F$_\lambda$ (10$^{-18}$ erg/s/cm$^2$/$\AA$)',size=20)
    plt.tick_params(axis='both', which='major', labelsize=15)
    plt.xlabel('Wavelength ($\AA$)',size=20)
    if save_plot == True:
        plt.savefig(plot_name)
    else:
        plt.show()
    plt.close()

In [33]:
%matplotlib inline

for i in galDB.index:
    Plot_with_phot(galDB['gids'][i],galDB['hi_res_specz'][i],galDB['Z'][i],galDB['t'][i],tau
                   ,save_plot=True, plot_name='../plots/phot_plots/%s_phot_sp.png' % galDB['gids'][i])

../data/galaxy_flts/n14713_flt: photutils.detect_sources (detect_thresh=0.0, grow_seg=5, gauss_fwhm=2.0, ZP=0.0)
../data/galaxy_flts/n14713_flt: photutils.source_properties
Rename column: ycentroid -> y_flt


Rename column: ra_icrs_centroid -> ra
Rename column: dec_icrs_centroid -> dec
Rename column: xcentroid -> x_flt
../data/galaxy_flts/n14713_flt: photutils.source_properties - 1 objects
../data/galaxy_flts/n14713_flt: save ../data/galaxy_flts/n14713_flt.detect_seg.fits, ../data/galaxy_flts/n14713_flt.detect.cat


../data/galaxy_flts/n16758_flt: photutils.detect_sources (detect_thresh=0.0, grow_seg=5, gauss_fwhm=2.0, ZP=0.0)
../data/galaxy_flts/n16758_flt: photutils.source_properties
Rename column: ycentroid -> y_flt
Rename column: ra_icrs_centroid -> ra
Rename column: dec_icrs_centroid -> dec
Rename column: xcentroid -> x_flt
../data/galaxy_flts/n16758_flt: photutils.source_properties - 1 objects
../data/galaxy_flts/n16758_flt: save ../data/galaxy_flts/n16758_flt.detect_seg.fits, ../data/galaxy_flts/n16758_flt.detect.cat


../data/galaxy_flts/n17070_flt: photutils.detect_sources (detect_thresh=0.0, grow_seg=5, gauss_fwhm=2.0, ZP=0.0)
../data/galaxy_flts/n17070_flt: photutils.source_properties
Rename column: ycentroid -> y_flt
Rename column: ra_icrs_centroid -> ra
Rename column: dec_icrs_centroid -> dec
Rename column: xcentroid -> x_flt
../data/galaxy_flts/n17070_flt: photutils.source_properties - 1 objects
../data/galaxy_flts/n17070_flt: save ../data/galaxy_flts/n17070_flt.detect_seg.fits, ../data/galaxy_flts/n17070_flt.detect.cat


../data/galaxy_flts/n19442_flt: photutils.detect_sources (detect_thresh=0.0, grow_seg=5, gauss_fwhm=2.0, ZP=0.0)
../data/galaxy_flts/n19442_flt: photutils.source_properties
Rename column: ycentroid -> y_flt
Rename column: ra_icrs_centroid -> ra
Rename column: dec_icrs_centroid -> dec
Rename column: xcentroid -> x_flt
../data/galaxy_flts/n19442_flt: photutils.source_properties - 1 objects
../data/galaxy_flts/n19442_flt: save ../data/galaxy_flts/n19442_flt.detect_seg.fits, ../data/galaxy_flts/n19442_flt.detect.cat


...

../data/galaxy_flts/s46066_flt: photutils.detect_sources (detect_thresh=0.0, grow_seg=5, gauss_fwhm=2.0, ZP=0.0)
../data/galaxy_flts/s46066_flt: photutils.source_properties


Rename column: ycentroid -> y_flt
Rename column: ra_icrs_centroid -> ra
Rename column: dec_icrs_centroid -> dec
Rename column: xcentroid -> x_flt
../data/galaxy_flts/s46066_flt: photutils.source_properties - 1 objects
../data/galaxy_flts/s46066_flt: save ../data/galaxy_flts/s46066_flt.detect_seg.fits, ../data/galaxy_flts/s46066_flt.detect.cat


../data/galaxy_flts/s46345_flt: photutils.detect_sources (detect_thresh=0.0, grow_seg=5, gauss_fwhm=2.0, ZP=0.0)
../data/galaxy_flts/s46345_flt: photutils.source_properties
Rename column: ycentroid -> y_flt
Rename column: ra_icrs_centroid -> ra
Rename column: dec_icrs_centroid -> dec
Rename column: xcentroid -> x_flt
../data/galaxy_flts/s46345_flt: photutils.source_properties - 1 objects
../data/galaxy_flts/s46345_flt: save ../data/galaxy_flts/s46345_flt.detect_seg.fits, ../data/galaxy_flts/s46345_flt.detect.cat


../data/galaxy_flts/s47677_flt: photutils.detect_sources (detect_thresh=0.0, grow_seg=5, gauss_fwhm=2.0, ZP=0.0)
../data/galaxy_flts/s47677_flt: photutils.source_properties
Rename column: ycentroid -> y_flt
Rename column: ra_icrs_centroid -> ra
Rename column: dec_icrs_centroid -> dec
Rename column: xcentroid -> x_flt
../data/galaxy_flts/s47677_flt: photutils.source_properties - 1 objects
../data/galaxy_flts/s47677_flt: save ../data/galaxy_flts/s47677_flt.detect_seg.fits, ../data/galaxy_flts/s47677_flt.detect.cat


../data/galaxy_flts/s48631_flt: photutils.detect_sources (detect_thresh=0.0, grow_seg=5, gauss_fwhm=2.0, ZP=0.0)
../data/galaxy_flts/s48631_flt: photutils.source_properties
Rename column: ycentroid -> y_flt
Rename column: ra_icrs_centroid -> ra
Rename column: dec_icrs_centroid -> dec
Rename column: xcentroid -> x_flt
../data/galaxy_flts/s48631_flt: photutils.source_properties - 2 objects
../data/galaxy_flts/s48631_flt: save ../data/galaxy_flts/s48631_flt.detect_seg.fits, ../data/galaxy_flts/s48631_flt.detect.cat


In [12]:
filter_names = ['F850LP','F098M','Subaru-IA827','Subaru-IA856','Subaru-suprime-z']
gewv = [9001,9841,8245,8563,9077]
gfwhm = [1235,1163,245,266,970]
gphot_db = pd.DataFrame({'eff_wv' : gewv, 'fwhm' : gfwhm },index = filter_names)

In [13]:
gphot_db

,eff_wv,fwhm
F850LP,9001,1235
F098M,9841,1163
Subaru-IA827,8245,245
Subaru-IA856,8563,266
Subaru-suprime-z,9077,970


In [15]:
filters = [240,201,197,198,284]

for i in galDB.index:
    spec = Gen_spec(galDB['gids'][i], galDB['hi_res_specz'][i])

    gphot = np.repeat(-99.,len(filters))
    gphoter = np.repeat(-99.,len(filters))
    gal_phot = Photometry(spec.gal_wv,spec.gal_fl,spec.gal_er,240)
    for ii in range(len(filters)):
        gal_phot.Get_Sensitivity(filters[ii])
        if spec.gal_wv[0] <= gal_phot.sens_wv[0] and gal_phot.sens_wv[-1] <= spec.gal_wv[-1]:
            gal_phot.Photo()
            effwv,gphot[ii],gphoter[ii] = [gal_phot.eff_wv,gal_phot.photo,gal_phot.photo_er]
    gphot_add = pd.DataFrame({galDB['gids'][i] : gphot, '%s_err' % galDB['gids'][i] : gphoter },index = filter_names)
    gphot_db = gphot_db.join(gphot_add)

../data/galaxy_flts/n14713_flt: photutils.detect_sources (detect_thresh=0.0, grow_seg=5, gauss_fwhm=2.0, ZP=0.0)
../data/galaxy_flts/n14713_flt: photutils.source_properties
Rename column: ycentroid -> y_flt
Rename column: ra_icrs_centroid -> ra
Rename column: dec_icrs_centroid -> dec
Rename column: xcentroid -> x_flt
../data/galaxy_flts/n14713_flt: photutils.source_properties - 1 objects


../data/galaxy_flts/n14713_flt: save ../data/galaxy_flts/n14713_flt.detect_seg.fits, ../data/galaxy_flts/n14713_flt.detect.cat
../data/galaxy_flts/n16758_flt: photutils.detect_sources (detect_thresh=0.0, grow_seg=5, gauss_fwhm=2.0, ZP=0.0)
../data/galaxy_flts/n16758_flt: photutils.source_properties
Rename column: ycentroid -> y_flt
Rename column: ra_icrs_centroid -> ra
Rename column: dec_icrs_centroid -> dec
Rename column: xcentroid -> x_flt
../data/galaxy_flts/n16758_flt: photutils.source_properties - 1 objects
../data/galaxy_flts/n16758_flt: save ../data/galaxy_flts/n16758_flt.detect_seg.fits, ../data/galaxy_flts/n16758_flt.detect.cat


../data/galaxy_flts/n17070_flt: photutils.detect_sources (detect_thresh=0.0, grow_seg=5, gauss_fwhm=2.0, ZP=0.0)
../data/galaxy_flts/n17070_flt: photutils.source_properties


Rename column: ycentroid -> y_flt
Rename column: ra_icrs_centroid -> ra
Rename column: dec_icrs_centroid -> dec
Rename column: xcentroid -> x_flt
../data/galaxy_flts/n17070_flt: photutils.source_properties - 1 objects
../data/galaxy_flts/n17070_flt: save ../data/galaxy_flts/n17070_flt.detect_seg.fits, ../data/galaxy_flts/n17070_flt.detect.cat


../data/galaxy_flts/n19442_flt: photutils.detect_sources (detect_thresh=0.0, grow_seg=5, gauss_fwhm=2.0, ZP=0.0)
../data/galaxy_flts/n19442_flt: photutils.source_properties
Rename column: ycentroid -> y_flt
Rename column: ra_icrs_centroid -> ra
Rename column: dec_icrs_centroid -> dec
Rename column: xcentroid -> x_flt
../data/galaxy_flts/n19442_flt: photutils.source_properties - 1 objects
../data/galaxy_flts/n19442_flt: save ../data/galaxy_flts/n19442_flt.detect_seg.fits, ../data/galaxy_flts/n19442_flt.detect.cat


...

../data/galaxy_flts/s46345_flt: photutils.detect_sources (detect_thresh=0.0, grow_seg=5, gauss_fwhm=2.0, ZP=0.0)
../data/galaxy_flts/s46345_flt: photutils.source_properties
Rename column: ycentroid -> y_flt
Rename column: ra_icrs_centroid -> ra
Rename column: dec_icrs_centroid -> dec
Rename column: xcentroid -> x_flt
../data/galaxy_flts/s46345_flt: photutils.source_properties - 1 objects
../data/galaxy_flts/s46345_flt: save ../data/galaxy_flts/s46345_flt.detect_seg.fits, ../data/galaxy_flts/s46345_flt.detect.cat


../data/galaxy_flts/s47677_flt: photutils.detect_sources (detect_thresh=0.0, grow_seg=5, gauss_fwhm=2.0, ZP=0.0)
../data/galaxy_flts/s47677_flt: photutils.source_properties


Rename column: ycentroid -> y_flt
Rename column: ra_icrs_centroid -> ra
Rename column: dec_icrs_centroid -> dec
Rename column: xcentroid -> x_flt
../data/galaxy_flts/s47677_flt: photutils.source_properties - 1 objects
../data/galaxy_flts/s47677_flt: save ../data/galaxy_flts/s47677_flt.detect_seg.fits, ../data/galaxy_flts/s47677_flt.detect.cat


../data/galaxy_flts/s48631_flt: photutils.detect_sources (detect_thresh=0.0, grow_seg=5, gauss_fwhm=2.0, ZP=0.0)
../data/galaxy_flts/s48631_flt: photutils.source_properties
Rename column: ycentroid -> y_flt
Rename column: ra_icrs_centroid -> ra
Rename column: dec_icrs_centroid -> dec
Rename column: xcentroid -> x_flt


../data/galaxy_flts/s48631_flt: photutils.source_properties - 2 objects
../data/galaxy_flts/s48631_flt: save ../data/galaxy_flts/s48631_flt.detect_seg.fits, ../data/galaxy_flts/s48631_flt.detect.cat


In [18]:
gphot_db

,eff_wv,fwhm,n14713,n14713_err,n16758,n16758_err,n17070,n17070_err,n19442,n19442_err,...,s45972,s45972_err,s46066,s46066_err,s46345,s46345_err,s47677,s47677_err,s48631,s48631_err
F850LP,9001,1235,-9.900000e+01,-9.900000e+01,1.747697e-18,2.422580e-19,1.529235e-18,4.663560e-19,3.264858e-19,3.000168e-19,...,2.011440e-18,2.151872e-19,-9.900000e+01,-9.900000e+01,-9.900000e+01,-9.900000e+01,5.076848e-19,2.006029e-19,1.673274e-18,3.372973e-19
F098M,9841,1163,9.769333e-19,2.634086e-19,2.098037e-18,1.445953e-19,1.996690e-18,2.641406e-19,3.732585e-19,2.511762e-19,...,2.273222e-18,1.341041e-19,-9.900000e+01,-9.900000e+01,2.392788e-19,1.000450e-19,6.410471e-19,1.298577e-19,1.888517e-18,1.678651e-19
Subaru-IA827,8245,245,8.229581e-19,8.340300e-19,1.344934e-18,4.807593e-19,1.026516e-18,9.274676e-19,3.103016e-19,5.189770e-19,...,1.572377e-18,4.653268e-19,5.621589e-19,6.620920e-19,2.536885e-19,3.105887e-19,3.113629e-19,3.917838e-19,1.490061e-18,9.242394e-19
Subaru-IA856,8563,266,-9.900000e+01,-9.900000e+01,-9.900000e+01,-9.900000e+01,1.064267e-18,6.100259e-19,2.494950e-19,3.569217e-19,...,1.782134e-18,2.815424e-19,6.039884e-19,4.402900e-19,-9.900000e+01,-9.900000e+01,3.919963e-19,2.617631e-19,1.533059e-18,5.284655e-19
Subaru-suprime-z,9077,970,7.964886e-19,3.818980e-19,1.796225e-18,2.212656e-19,1.541097e-18,4.311053e-19,3.083753e-19,2.809899e-19,...,2.043646e-18,1.945843e-19,6.294148e-19,2.951754e-19,2.333093e-19,1.460643e-19,5.240084e-19,1.831599e-19,1.641221e-18,2.742698e-19


In [22]:
gphot_db.to_pickle('../data/gphot_db.pkl')